# 09_7th_test 요약

- 모델: FT-Transformer + Optuna(가능 시)
- 라벨: voted==2 를 양성으로 학습, 제출은 P(voted==2) 확률
- 피처: age_group_ord, education, reliability_score, total_time_log, tp_emo, cat: race/religion/age_edu/married_cat
- 학습/평가: train/val 8:2 stratified split, StandardScaler + LabelEncoder, ROC-AUC 기준
- 제출파일: submission_prob_7th_test.csv


## 1. 라이브러리 로드


In [ ]:
import os
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score


# 공통 하이퍼파라미터



## 2. 설정값 및 시드 고정


In [ ]:
SEED = 42
BATCH_SIZE = 256
EPOCHS = 50
PATIENCE = 6
OPTUNA_TRIALS = 50  # 필요 시 조절

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)



## 3. 데이터 로드


In [ ]:
train_raw = pd.read_csv("../../data/raw/train.csv")
test_raw  = pd.read_csv("../../data/raw/test_x.csv")



## 4. 피처 생성 함수


In [ ]:
# -----------------------------
# 공통 전처리/피처 생성 함수
# -----------------------------
def build_features(df_raw, cfg=None, is_train=True):
    df = df_raw.copy()

    # 0) voted -> voted_bin (train only)
    if is_train and "voted" in df.columns:
        df["voted_bin"] = (df["voted"] == 2).astype(int)

    # 1) age_group -> ordinal
    if "age_group" in df.columns:
        age_map = {"10s":1, "20s":2, "30s":3, "40s":4, "50s":5, "60s":6, "+70s":7}
        df["age_group_ord"] = df["age_group"].map(age_map).astype("float32")

    # 2) education (0 -> NaN -> train mean)
    if "education" in df.columns:
        df["education"] = pd.to_numeric(df["education"], errors="coerce")
        df.loc[df["education"] == 0, "education"] = np.nan
        if cfg is not None:
            df["education"] = df["education"].fillna(cfg["education_mean"])
        df["education"] = df["education"].astype("float32")

    # 3) familysize (clip + numeric)
    if "familysize" in df.columns:
        df["familysize"] = pd.to_numeric(df["familysize"], errors="coerce")
        df["familysize"] = df["familysize"].clip(upper=15).astype("float32")

    # -----------------------------
    # (A) Machiavellianism: mach_score / mach_std
    # -----------------------------
    # QaA ~ QtA 전체 사용 (0은 무응답으로 NaN 처리)
    qa_cols = [f"Q{c}A" for c in "abcdefghijklmnopqrst"]  # 20개
    exist_qa = [c for c in qa_cols if c in df.columns]

    # reverse scoring (Gemini v2 가이드)
    reverse_cols = ['QaA', 'QdA', 'QgA', 'QiA', 'QlA', 'QnA', 'QpA', 'QrA', 'QtA']
    for col in exist_qa:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df.loc[df[col] == 0, col] = np.nan
        if col in reverse_cols:
            df[col] = 6 - df[col]

    if len(exist_qa) >= 10:  # 최소한 좀 있어야 의미 있음
        df["mach_score"] = df[exist_qa].mean(axis=1).astype("float32")
        df["mach_std"]   = df[exist_qa].std(axis=1).astype("float32")

    # -----------------------------
    # (B) Q_E time: total_time_log
    # -----------------------------
    qe_cols = [f"Q{c}E" for c in "abcdefghijklmnopqrst"]
    exist_qe = [c for c in qe_cols if c in df.columns]
    for col in exist_qe:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df.loc[df[col] < 0, col] = np.nan
        df[col] = np.log1p(df[col].clip(lower=0, upper=1e6))  # log1p + clip

    if len(exist_qe) >= 10:
        df["total_time_log"] = df[exist_qe].sum(axis=1).astype("float32")

    # -----------------------------
    # (C) Word reliability: reliability_score
    # -----------------------------
    wr_cols = [f"wr_{i:02d}" for i in range(1, 14)]
    wf_cols = [f"wf_{i:02d}" for i in range(1, 4)]
    exist_wr = [c for c in wr_cols if c in df.columns]
    exist_wf = [c for c in wf_cols if c in df.columns]

    for col in exist_wr + exist_wf:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    if len(exist_wr) == 13:
        df["wr_sum"] = df[exist_wr].sum(axis=1).astype("float32")
    if len(exist_wf) == 3:
        df["wf_sum"] = df[exist_wf].sum(axis=1).astype("float32")

    if "wr_sum" in df.columns and "wf_sum" in df.columns:
        df["reliability_score"] = (df["wr_sum"] - (df["wf_sum"] * 2)).astype("float32")

    # -----------------------------
    # (D) Big5: tp_extra / tp_agree / tp_cons / tp_emo / tp_open
    # -----------------------------
    # 7점 척도 역채점: 8 - x
    def _num(s): 
        return pd.to_numeric(s, errors="coerce")

    if "tp01" in df.columns and "tp06" in df.columns:
        df["tp_extra"] = ((_num(df["tp01"]) + (7 - _num(df["tp06"]))) / 2).astype("float32")
    if "tp02" in df.columns and "tp07" in df.columns:
        df["tp_agree"] = (((7 - _num(df["tp02"])) + _num(df["tp07"])) / 2).astype("float32")
    if "tp03" in df.columns and "tp08" in df.columns:
        df["tp_cons"]  = ((_num(df["tp03"]) + (7 - _num(df["tp08"]))) / 2).astype("float32")
    if "tp04" in df.columns and "tp09" in df.columns:
        df["tp_emo"]   = (((7 - _num(df["tp04"])) + _num(df["tp09"])) / 2).astype("float32")
    if "tp05" in df.columns and "tp10" in df.columns:
        df["tp_open"]  = ((_num(df["tp05"]) + (7 - _num(df["tp10"]))) / 2).astype("float32")

    # -----------------------------
    # Categorical: race / religion / age_edu / urban / gender / married / hand
    # -----------------------------
    # married_cat / hand_cat (0 -> NaN -> string)
    if "married" in df.columns:
        df["married"] = pd.to_numeric(df["married"], errors="coerce")
        df.loc[df["married"] == 0, "married"] = np.nan
        df["married_cat"] = df["married"].astype("string")

    if "hand" in df.columns:
        df["hand"] = pd.to_numeric(df["hand"], errors="coerce")
        df.loc[df["hand"] == 0, "hand"] = np.nan
        df["hand_cat"] = df["hand"].astype("string")

    # urban_cat / gender_cat: 원본을 카테고리로 유지
    if "urban" in df.columns:
        df["urban"] = pd.to_numeric(df["urban"], errors="coerce")
        df.loc[df["urban"] == 0, "urban"] = np.nan
        df["urban_cat"] = df["urban"].astype("string")

    if "gender" in df.columns:
        df["gender"] = pd.to_numeric(df["gender"], errors="coerce")
        df.loc[df["gender"] == 0, "gender"] = np.nan
        df["gender_cat"] = df["gender"].astype("string")

    # age_edu: age_group + "_" + education
    if "age_group" in df.columns and "education" in df.columns:
        edu_str = df["education"].fillna(-1).round(0).astype("int").astype(str)
        df["age_edu"] = df["age_group"].astype(str) + "_" + edu_str

    return df



target = "voted_bin"



## 5. 피처 리스트


In [ ]:
feature_cols = [
    # NUM
    "age_group_ord", "education",
    "reliability_score", "total_time_log", "tp_emo",
    # CAT
    "race", "religion", "age_edu", "married_cat",
]

cat_cols = ["race", "religion", "age_edu", "married_cat"]



## 6. 전처리 함수 (인코딩/스케일링)


In [ ]:
def _normalize_cat(s: pd.Series) -> pd.Series:
    s = s.astype("string")
    s = s.fillna("__MISSING__")
    s = s.replace({"<NA>": "__MISSING__", "nan": "__MISSING__", "NaN": "__MISSING__"})
    return s


def _fit_label_encoders(df_train, cat_cols):
    encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        s = _normalize_cat(df_train[col])
        # __MISSING__ 토큰 보장
        if "__MISSING__" not in s.unique():
            s = pd.concat([s, pd.Series(["__MISSING__"])], ignore_index=True)
        le.fit(s.astype(str))
        encoders[col] = le
    return encoders


def _transform_with_encoder(series, le: LabelEncoder):
    s = _normalize_cat(series)
    classes = set(le.classes_)
    s = s.apply(lambda x: x if x in classes else "__MISSING__")
    return le.transform(s.astype(str))


def preprocess_split_fit_transform(df_train, df_val, df_test, feature_cols, cat_cols, target):
    # target
    y_train = df_train[target].values
    y_val = df_val[target].values

    # 인코더 fit (train only)
    encoders = _fit_label_encoders(df_train, cat_cols)

    # categorical transform
    X_cat_train = np.stack([_transform_with_encoder(df_train[c], encoders[c]) for c in cat_cols], axis=1)
    X_cat_val   = np.stack([_transform_with_encoder(df_val[c], encoders[c])   for c in cat_cols], axis=1)
    X_cat_test  = np.stack([_transform_with_encoder(df_test[c], encoders[c])  for c in cat_cols], axis=1)

    # numeric
    num_cols = [c for c in feature_cols if c not in cat_cols]
    train_num = df_train[num_cols].apply(pd.to_numeric, errors="coerce")
    val_num   = df_val[num_cols].apply(pd.to_numeric, errors="coerce")
    test_num  = df_test[num_cols].apply(pd.to_numeric, errors="coerce")

    train_means = train_num.mean()
    train_num = train_num.fillna(train_means)
    val_num   = val_num.fillna(train_means)
    test_num  = test_num.fillna(train_means)

    scaler = StandardScaler()
    X_num_train = scaler.fit_transform(train_num.values)
    X_num_val   = scaler.transform(val_num.values)
    X_num_test  = scaler.transform(test_num.values)

    return {
        "X_num_train": X_num_train,
        "X_num_val": X_num_val,
        "X_num_test": X_num_test,
        "X_cat_train": X_cat_train,
        "X_cat_val": X_cat_val,
        "X_cat_test": X_cat_test,
        "y_train": y_train,
        "y_val": y_val,
        "encoders": encoders,
        "scaler": scaler,
        "num_cols": num_cols,
    }


# raw에서 먼저 split
raw_y = (train_raw["voted"] == 2).astype(int)



## 7. Train/Val 분리 및 피처 생성


In [ ]:
train_raw_split, val_raw_split = train_test_split(
    train_raw,
    test_size=0.2,
    random_state=SEED,
    stratify=raw_y
)

# cfg는 train split 기준으로만 계산
cfg = {}
edu = pd.to_numeric(train_raw_split["education"], errors="coerce").replace(0, np.nan)
cfg["education_mean"] = float(edu.mean())

cfg["race_majors"] = set(train_raw_split["race"].value_counts(dropna=True).head(5).index)
cfg["religion_majors"] = set(train_raw_split["religion"].value_counts(dropna=True).head(5).index)

# build_features
train_feat = build_features(train_raw_split, cfg=cfg, is_train=True)
val_feat   = build_features(val_raw_split,   cfg=cfg, is_train=True)

test_feat  = build_features(test_raw, cfg=cfg, is_train=False)


prep = preprocess_split_fit_transform(
    train_feat, val_feat, test_feat,
    feature_cols=feature_cols,
    cat_cols=cat_cols,
    target=target
)

X_num_train = prep["X_num_train"]
X_num_val   = prep["X_num_val"]
X_num_test  = prep["X_num_test"]
X_cat_train = prep["X_cat_train"]
X_cat_val   = prep["X_cat_val"]
X_cat_test  = prep["X_cat_test"]

y_train = prep["y_train"]
y_val   = prep["y_val"]

num_cols = prep["num_cols"]



## 8. Dataset & 모델 정의


In [ ]:
class TabDataset(Dataset):
    def __init__(self, X_num, X_cat, y=None):
        self.X_num = torch.tensor(X_num, dtype=torch.float32)
        self.X_cat = torch.tensor(X_cat, dtype=torch.long)
        self.y = None if y is None else torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.X_num)

    def __getitem__(self, idx):
        if self.y is None:
            return self.X_num[idx], self.X_cat[idx]
        return self.X_num[idx], self.X_cat[idx], self.y[idx]


class NumericalEmbedding(nn.Module):
    def __init__(self, num_features, d_token):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(num_features, d_token))
        self.bias = nn.Parameter(torch.zeros(num_features, d_token))

    def forward(self, x):
        x = x.unsqueeze(-1)  # (B, num_features, 1)
        return x * self.weight + self.bias

class FTTransformer(nn.Module):
    def __init__(self, cat_dims, num_features, d_token=64, n_layers=2, n_heads=4,
                 dropout=0.2, attn_dropout=0.2):
        super().__init__()
        self.cat_embeds = nn.ModuleList([nn.Embedding(dim, d_token) for dim in cat_dims])
        self.num_embed = NumericalEmbedding(num_features, d_token)

        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_token))

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_token,
            nhead=n_heads,
            dim_feedforward=d_token * 4,
            dropout=attn_dropout,
            batch_first=True,
            activation='gelu'
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(d_token),
            nn.Linear(d_token, d_token),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_token, 1)
        )

    def forward(self, x_num, x_cat):
        cat_tokens = []
        for i, emb in enumerate(self.cat_embeds):
            cat_tokens.append(emb(x_cat[:, i]))
        cat_tokens = torch.stack(cat_tokens, dim=1) if len(cat_tokens) > 0 else None

        num_tokens = self.num_embed(x_num)
        tokens = num_tokens if cat_tokens is None else torch.cat([cat_tokens, num_tokens], dim=1)

        cls = self.cls_token.expand(tokens.size(0), -1, -1)
        tokens = torch.cat([cls, tokens], dim=1)

        x = self.transformer(tokens)
        cls_out = x[:, 0]
        return self.head(cls_out)



## 9. 학습/평가 함수


In [ ]:
def train_one_model(model, train_loader, val_loader, y_val, device, epochs=EPOCHS, patience=PATIENCE):
    model.to(device)

    pos_weight = (len(y_train) - y_train.sum()) / (y_train.sum() + 1e-6)
    pos_weight_t = torch.tensor([pos_weight], dtype=torch.float32, device=device)

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=3e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-5
    )

    best_auc = -1.0
    best_state = None
    patience_ctr = 0

    for epoch in range(epochs):
        model.train()
        for xb_num, xb_cat, yb in train_loader:
            xb_num = xb_num.to(device)
            xb_cat = xb_cat.to(device)
            yb = yb.to(device)

            optimizer.zero_grad()
            logits = model(xb_num, xb_cat)
            loss = criterion(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        # val AUC
        model.eval()
        val_probs = []
        with torch.no_grad():
            for xb_num, xb_cat, _ in val_loader:
                xb_num = xb_num.to(device)
                xb_cat = xb_cat.to(device)
                logits = model(xb_num, xb_cat)
                probs = torch.sigmoid(logits).cpu().numpy().ravel()
                val_probs.append(probs)

        val_probs = np.concatenate(val_probs)
        val_auc = roc_auc_score(y_val, val_probs)
        scheduler.step(val_auc)

        if val_auc > best_auc + 1e-4:
            best_auc = val_auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience_ctr = 0
        else:
            patience_ctr += 1

        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1} | val AUC {val_auc:.4f}")

        if patience_ctr >= patience:
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_auc


def predict_proba(model, loader, device):
    model.eval()
    probs = []
    with torch.no_grad():
        for xb_num, xb_cat in loader:
            xb_num = xb_num.to(device)
            xb_cat = xb_cat.to(device)
            logits = model(xb_num, xb_cat)
            p = torch.sigmoid(logits).cpu().numpy().ravel()
            probs.append(p)
    return np.concatenate(probs)



## 10. Optuna 튜닝


In [ ]:
try:
    import optuna
    from optuna.pruners import MedianPruner
    OPTUNA_AVAILABLE = True
except Exception as e:
    OPTUNA_AVAILABLE = False
    print("Optuna not available:", e)

best_params = {
    "d_token": 64,
    "n_layers": 2,
    "n_heads": 4,
    "dropout": 0.2,
    "attn_dropout": 0.2,
    "lr": 3e-4,
    "weight_decay": 3e-4,
}

if OPTUNA_AVAILABLE:
    def objective(trial):
        set_seed(SEED)

        d_token = trial.suggest_categorical("d_token", [32, 64, 96, 128])
        n_heads = trial.suggest_categorical("n_heads", [2, 4, 8])
        if d_token % n_heads != 0:
            raise optuna.exceptions.TrialPruned()

        n_layers = trial.suggest_int("n_layers", 1, 3)
        dropout = trial.suggest_float("dropout", 0.1, 0.4)
        attn_dropout = trial.suggest_float("attn_dropout", 0.1, 0.4)
        lr = trial.suggest_float("lr", 1e-4, 8e-4, log=True)
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 5e-4, log=True)

        # trial용 split
        raw_y = (train_raw["voted"] == 2).astype(int)
        tr_raw, va_raw = train_test_split(
            train_raw,
            test_size=0.2,
            random_state=SEED,
            stratify=raw_y
        )

        cfg = {}
        edu = pd.to_numeric(tr_raw["education"], errors="coerce").replace(0, np.nan)
        cfg["education_mean"] = float(edu.mean())
        cfg["race_majors"] = set(tr_raw["race"].value_counts(dropna=True).head(5).index)
        cfg["religion_majors"] = set(tr_raw["religion"].value_counts(dropna=True).head(5).index)

        tr_feat = build_features(tr_raw, cfg=cfg, is_train=True)
        va_feat = build_features(va_raw, cfg=cfg, is_train=True)
        te_feat = build_features(test_raw, cfg=cfg, is_train=False)

        prep_local = preprocess_split_fit_transform(
            tr_feat, va_feat, te_feat,
            feature_cols=feature_cols,
            cat_cols=cat_cols,
            target=target
        )

        X_num_tr = prep_local["X_num_train"]
        X_cat_tr = prep_local["X_cat_train"]
        y_tr = prep_local["y_train"]

        X_num_va = prep_local["X_num_val"]
        X_cat_va = prep_local["X_cat_val"]
        y_va = prep_local["y_val"]

        train_ds = TabDataset(X_num_tr, X_cat_tr, y_tr)
        val_ds = TabDataset(X_num_va, X_cat_va, y_va)
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)



## 11. 학습 실행 및 제출


In [ ]:
cat_dims = [len(prep_local["encoders"][c].classes_) for c in cat_cols]
        model = FTTransformer(
            cat_dims=cat_dims,
            num_features=prep_local["num_cols"].__len__(),
            d_token=d_token,
            n_layers=n_layers,
            n_heads=n_heads,
            dropout=dropout,
            attn_dropout=attn_dropout
        )

        model.to(DEVICE)
        pos_weight = (len(y_tr) - y_tr.sum()) / (y_tr.sum() + 1e-6)
        pos_weight_t = torch.tensor([pos_weight], dtype=torch.float32, device=DEVICE)

        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_t)
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

        best_auc = -1.0
        patience = 3
        patience_ctr = 0

        for epoch in range(20):
            model.train()
            for xb_num, xb_cat, yb in train_loader:
                xb_num = xb_num.to(DEVICE)
                xb_cat = xb_cat.to(DEVICE)
                yb = yb.to(DEVICE)

                optimizer.zero_grad()
                logits = model(xb_num, xb_cat)
                loss = criterion(logits, yb)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()

            # val auc
            model.eval()
            val_probs = []
            with torch.no_grad():
                for xb_num, xb_cat, _ in val_loader:
                    xb_num = xb_num.to(DEVICE)
                    xb_cat = xb_cat.to(DEVICE)
                    logits = model(xb_num, xb_cat)
                    probs = torch.sigmoid(logits).cpu().numpy().ravel()
                    val_probs.append(probs)

            val_probs = np.concatenate(val_probs)
            val_auc = roc_auc_score(y_va, val_probs)
            trial.report(val_auc, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

            if val_auc > best_auc + 1e-4:
                best_auc = val_auc
                patience_ctr = 0
            else:
                patience_ctr += 1

            if patience_ctr >= patience:
                break

        return best_auc

    study = optuna.create_study(direction="maximize", pruner=MedianPruner(n_warmup_steps=5))
    study.optimize(objective, n_trials=OPTUNA_TRIALS)
    best_params = study.best_params
    print("Optuna best params:", best_params)


cat_dims = [len(prep["encoders"][c].classes_) for c in cat_cols]

model = FTTransformer(
    cat_dims=cat_dims,
    num_features=len(num_cols),
    d_token=best_params.get("d_token", 64),
    n_layers=best_params.get("n_layers", 2),
    n_heads=best_params.get("n_heads", 4),
    dropout=best_params.get("dropout", 0.2),
    attn_dropout=best_params.get("attn_dropout", 0.2)
)

train_ds = TabDataset(X_num_train, X_cat_train, y_train)
val_ds = TabDataset(X_num_val, X_cat_val, y_val)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)

model, best_auc = train_one_model(model, train_loader, val_loader, y_val, DEVICE)
print(f"Best Validation ROC-AUC: {best_auc:.4f}")


test_ds = TabDataset(X_num_test, X_cat_test, y=None)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

test_probs = predict_proba(model, test_loader, DEVICE)

sub = pd.read_csv("../../data/raw/sample_submission.csv")
sub["voted"] = test_probs
sub.to_csv("submission_prob_7th_test.csv", index=False)
print("saved submission_prob_7th_test.csv")

